### Prepare Notebook 

In [4]:
!pip install azureml[automl]

  azureml 0.2.7 does not provide the extra 'automl'
You are using pip version 19.0.1, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
demo_ip='52.183.42.101'
subscription_id = '2a779d6f-0806-4359-a6e8-f1fd57bb5dd7'
resource_group  = 'scottgu-all-hands'
workspace_name  = 'contosomanufacturing'
experiment_name = 'contoso-regression'
project_folder = './contoso-regression'
model_url = "https://contosoautodatalake.dfs.core.windows.net/powerbi/AIDemos/AMLDataflow/model.json"
cluster_name = "cpucluster"
aks_cluster_name = 'contoso-aks'
aks_service_name ='contoso-service'
run_id = 'AutoML_f5cb69ce-68ef-4fbe-bdfb-928d55601d10' 

# Predicting Automotive Failures with Automated Azure Machine Learning 

<img align="left" width="1000" src="https://raw.githubusercontent.com/chrislauren/amlallhands/master/automl%20overview%201.png">

### Import open source Python packages

In [2]:
import logging
import os
import random
import re
import lightgbm
import pandas as pd
import numpy as np
import json
import csv
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
from sklearn import datasets
from shutil import copy2

### Import Azure Machine Learning Python SDK 

In [8]:
import azureml.core
from azureml.core import Workspace
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails

ModuleNotFoundError: No module named 'azureml.train.automl'

In [14]:
import CdmModel
from utils import read_from_adls_with_cdm_format, read_from_adls, generate_aad_token

### Connect to Azure Workspace

In [9]:
try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    ws.write_config()
    print('Library configuration succeeded')
except:
    print('Workspace not found')

Performing interactive authentication. Please follow the instructions on the terminal.


Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.
Wrote the config file config.json to: c:\Users\clauren\Downloads\ScottGu All-Hands\connectedcar\aml_config\config.json
Library configuration succeeded


### Create an Azure Machine Learning Experiment

In [10]:
ws = Workspace.from_config()

import os

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=output, index=['']).T

from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)

experiment = Experiment(ws, experiment_name)

Found the config file in: c:\Users\clauren\Downloads\ScottGu All-Hands\connectedcar\aml_config\config.json


### Fetch data from Azure Data Lake using Common Data Model

In [17]:
aad_token = generate_aad_token()
model_data = read_from_adls(endpoint = model_url, auth = aad_token)
model = CdmModel.Model.fromJson(model_data.json())

train_data = read_from_adls_with_cdm_format(model.entities[0])

In [21]:
train_data.head(10)

,Survival_In_Days,Province,Region,Trip_Length_Mean,Trip_Length_Sigma,Trips_Per_Day_Mean,Trips_Per_Day_Sigma,Battery_Rated_Cycles,Manufacture_Month,Manufacture_Year,...,Sensor_Reading_52,Sensor_Reading_53,Sensor_Reading_54,Sensor_Reading_55,Sensor_Reading_56,Sensor_Reading_57,Sensor_Reading_58,Sensor_Reading_59,Sensor_Reading_60,Sensor_Reading_61
0,1283,Bretagne,West,18.10325,6.034416,4.733162,1.183291,275,M8,Y2010,...,16.418910,17.441310,24.71829,11.812310,19.437210,15.079740,16.982440,18.893610,13.59000,14.510940
1,1427,Occitanie,South,14.63707,4.879023,4.325950,1.081487,250,M8,Y2014,...,14.703280,16.154500,27.78955,22.292230,29.158610,21.739530,23.830780,19.480210,10.26412,18.009700
2,1436,Auvergne_Rhone_Alpes,South,14.50564,4.835215,4.418737,1.104684,250,M9,Y2018,...,22.389700,21.834420,28.74326,26.313940,15.589060,15.317560,19.613730,28.397800,19.80799,15.425770
3,894,Martinique,West,20.85052,6.950172,4.284968,1.071242,200,M10,Y2003,...,2.794836,13.993500,15.52458,6.298875,11.355190,14.396860,2.890394,6.362495,10.91607,10.004320
4,1539,Reunion,South,11.57959,3.859862,4.561532,1.140383,200,M10,Y2007,...,26.631860,26.116980,18.01190,25.257760,25.320780,26.894640,18.863220,25.744930,24.02772,23.657220
5,1872,Marseille,South,14.07098,4.690325,4.697100,1.174275,300,M11,Y2011,...,11.889280,7.358676,10.70027,8.218617,13.397930,2.973648,11.031080,3.532511,12.84172,8.153067
6,151,Ile_de_France,MidWest,13.38851,4.462836,4.539887,1.134972,300,M12,Y2015,...,-13.361990,12.716510,-25.99962,-0.855164,-19.726040,5.154581,-9.921854,-0.260530,-20.26094,6.349902
7,1975,Normandie,MidWest,16.71867,5.572891,4.641222,1.160305,275,M12,Y2000,...,21.916700,14.228060,11.37833,-0.157791,13.303480,7.164655,10.716000,4.709601,19.31674,-0.762613
8,1957,Paris,MidWest,12.28045,4.093483,4.417785,1.104446,275,M1,Y2005,...,-10.007310,5.053398,-12.77030,4.404034,-8.639040,3.357793,-12.641930,-0.040223,-11.43332,4.471225
9,1150,Corse,South,19.61572,6.538573,4.318250,1.079563,250,M2,Y2009,...,17.566970,0.334354,11.38173,-9.591447,9.592852,-2.911261,8.209939,-2.191684,12.87396,-3.348420


### Create AML Compute Cluster

In [11]:
### Create AML CPU Compute Cluster

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

Found existing compute target.


### Create Azure Machine Learning Automated ML config

In [26]:
automl_config = AutoMLConfig(task = 'regression',
                             iterations = 20,
                             iteration_timeout_minutes = 10, 
                             max_cores_per_iteration = 10,
                             preprocess = True,
                             primary_metric = 'normalized_root_mean_squared_error',
                             n_cross_validations = 5,
                             debug_log = 'automl.log',
                             verbosity = logging.DEBUG,
                             data_script = project_folder + "/get_data.py",
                             compute_target = compute_target,
                             #blacklist_models = "",
                             path = project_folder)

<img align="left" width="1000" src="https://raw.githubusercontent.com/chrislauren/amlallhands/master/automl%20overview.png">

### Run our Experiment on AML Compute

In [27]:
remote_run = experiment.submit(automl_config, show_output=False)

### Display Automated ML Run Details

In [48]:
experiment = Experiment(ws, experiment_name)
ml_run = AutoMLRun(experiment=experiment, run_id=run_id)

In [49]:
RunDetails(ml_run).show() 

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 'sd…

### Show best run and model

In [ ]:
best_run, fitted_model = ml_run.get_output()


### Deploy the Model to AKS

<img align="left" width="1000" src="https://raw.githubusercontent.com/chrislauren/amlallhands/master/automl%20overview%202.png">

In [ ]:
# fetch the best model
best_run.download_file("outputs/model.pkl",
                       output_file_path = "./model.pkl")

In [ ]:
# register the model for deployment
model = Model.register(model_path = "model.pkl",
                       model_name = "model.pkl",
                       tags = {'area': "auto", 'type': "regression"},
                       description = "Contoso Auto model to predict battery failure",
                       workspace = ws)

print(model.name, model.description, model.version)

### Create Container Image

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  schema_file = "schema.json",
                                                  description = "Image with regression model",
                                                  tags = {'area': "auto", 'type': "regression"}
                                                 )

image = ContainerImage.create(name = "contosoimage",
                              # this is the model object
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

In [ ]:
print(model.name, model.description, model.version)

### Create/connect to the Kubernetes compute cluster

In [ ]:
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration(location='eastus2')

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                          name = aks_cluster_name, 
                          provisioning_configuration = prov_config)

aks_target.wait_for_completion(True)

### Deploy the model to Kubernetes

In [ ]:
aks_config = AksWebservice.deploy_configuration(collect_model_data=True, enable_app_insights=True)
aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target
                                           )
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

### Connect to the deployed webservice

In [28]:
aks_service = Webservice(ws,aks_service_name)

### Prepare production data and use the service
Note the call to aks_service.run() to test the model

In [29]:
import json
import csv

# fetch the test data
aad_token = generate_aad_token()
model_data = read_from_adls(endpoint = model_url,
                            auth = aad_token)
model = CdmModel.Model.fromJson(model_data.json())

test_data = read_from_adls_with_cdm_format(model.entities[1])

# prepare the test data
test_df = test_data.drop(columns=["Car_ID", "Battery_Age"])
    
# run the prediction
results = []
for index, row in test_df.iterrows():
    input_data = bytes(row.to_json(), encoding = 'utf8')
    prediction_data = aks_service.run(input_data = input_data)
    results.append(prediction_data["result"])

### Take the prediction and calculate which batteries need testing

In [39]:
result_columns = ['Car_ID','Predicted_Days_Remaining', 'Index']
result_df = pd.DataFrame(columns=result_columns)

for i in range(len(results)):
    car_ID = test_data['Car_ID'].loc[i]
    days_remaining = abs(results[i] - test_data['Battery_Age'].loc[i])
    if days_remaining < 400:
        car_values = [car_ID, days_remaining, i]
        result_df.loc[result_df.shape[0],:] = car_values

print('Percentage of cars that need batteries testing: {}%'.format(result_df.shape[0] / len(results) * 100))

Percentage of cars that need batteries testing: 30.0%


### Show example prediction data

In [40]:
result_df

,Car_ID,Predicted_Days_Remaining,Index
0,935039,359.935,0
1,707595,338.07,1
2,252240,347.662,7
